In [2]:
import pandas as pd
from time import *
from pprint import pprint

## mud

In [3]:
mud = pd.read_csv("/home/zhx/word/DriverOrderOfflineRL/mud_new", sep='\t', dtype='object')
mud.columns = [c.split(".")[-1] for c in mud.columns]
mud = mud[mud['order_id'].apply(lambda x:len(str(x)))==19]
mud = mud.fillna(-1)

In [4]:
mud.dtypes

order_id      object
maxaction     object
max_qvalue    object
action        object
qvalues       object
waittime      object
ymd           object
rn            object
dtype: object

In [5]:
mud.head()

,order_id,maxaction,max_qvalue,action,qvalues,waittime,ymd,rn
0,1441719259641590704,2.210000,0.373863,2.210000,"0.321462,0.345300,0.334358,0.319799,0.364069,0...",205,20231101,1
1,1441719259641590704,1.583000,0.353496,1.583000,"0.322574,0.323923,0.305968,0.324037,0.353496,0...",207,20231101,2
2,1441719259641590704,1.583000,0.375224,1.583000,"0.324030,0.296670,0.289612,0.326526,0.375224,0...",209,20231101,3
3,1441719259641590704,2.210000,0.383570,2.210000,"0.287638,0.301849,0.360628,0.338752,0.294719,0...",211,20231101,4
4,1441719259641590704,1.583000,0.462522,1.583000,"0.274647,0.279902,0.195810,0.278291,0.462522,0...",213,20231101,5


In [6]:
mud['order_id'] = mud['order_id'].astype('int')
mud[mud.columns.difference(['order_id', 'qvalues'])] = mud[mud.columns.difference(['order_id', 'qvalues'])].astype(float)

## feature

In [7]:
feature = pd.read_csv("/home/zhx/word/DriverOrderOfflineRL/feature_new", sep='\t',dtype='object')
feature.columns = [c.split(".")[-1] for c in feature.columns]
feature = feature[feature['f_real_order_id'].apply(lambda x:len(str(x)))==19]
feature = feature.fillna(-1)

In [8]:
feature.shape

(1226246, 434)

In [9]:
feature.drop(columns=['time_stamp', 'createtimestamp', 'f_trace_id', 'b_trace_id', 'm_trace_id'], inplace=True)

In [10]:
id_columns = ['f_real_order_id', 'f_order_id', 'b_real_order_id', 'driver_id', 'm_real_order_id']

feature[id_columns] = feature[id_columns].astype(int)
feature[feature.columns.difference(id_columns)] = feature[feature.columns.difference(id_columns)].astype(float)

In [12]:
feature['f_order_id'].value_counts()

f_order_id
87996125500460    255
87996128863879    239
87996132060623    233
87996128495030    230
87996108863677    224
                 ... 
87996147654116      1
87996115653544      1
87996108918827      1
87996123105122      1
87996111247457      1
Name: count, Length: 212352, dtype: int64

In [13]:
feature.dtypes

f_real_order_id                  int64
f_order_id                       int64
nearby_total_driver_num        float64
nearby_total_order_num         float64
nearby_idle_driver_num         float64
                                ...   
driver_dynamic_times           float64
decision_time                  float64
decision_dropoff2order_time    float64
m_real_order_id                  int64
waittime                       float64
Length: 429, dtype: object

In [14]:
feature[feature['f_order_id'] == 87996125500460][['f_real_order_id', 'f_order_id', 'b_real_order_id']]

,f_real_order_id,f_order_id,b_real_order_id
2859,1441719259658802220,87996125500460,-1
13881,1441719259658802220,87996125500460,-1
14514,1441719259658802220,87996125500460,-1
28681,1441719259658802220,87996125500460,-1
32819,1441719259658802220,87996125500460,-1
...,...,...,...
1184129,1441719259658802220,87996125500460,-1
1203116,1441719259658802220,87996125500460,-1
1209843,1441719259658802220,87996125500460,-1
1211595,1441719259658802220,87996125500460,-1


## order

In [15]:
order = pd.read_csv("/home/zhx/word/DriverOrderOfflineRL/order_new", sep='\t', dtype='object')
order.columns = [c.split(".")[-1] for c in order.columns]
order = order[order['order_id'].apply(lambda x:len(str(x)))==14]
order = order.fillna(-1)

In [16]:
order.shape

(97387, 13)

In [21]:
order.dtypes

order_id                                               int64
driver_id                                              int64
calls                                                  int64
rides                                                  int64
acceptances                                            int64
calls_cancelled_before_accepted                        int64
calls_cancelled_by_passenger_after_accepted            int64
calls_cancelled_by_driver_after_accepted               int64
calls_cancelled_by_drivers_after_accepted_addresend    int64
serial_calls                                           int64
serial_rides                                           int64
serial_acceptances                                     int64
serial_pcaa_cnt                                        int64
dtype: object

In [18]:
order.head(100)

,order_id,driver_id,calls,rides,acceptances,calls_cancelled_before_accepted,calls_cancelled_by_passenger_after_accepted,calls_cancelled_by_driver_after_accepted,calls_cancelled_by_drivers_after_accepted_addresend,serial_calls,serial_rides,serial_acceptances,serial_pcaa_cnt
0,87996129015125,0,1,0,0,1,0,0,0,0,0,0,0
1,87996138538853,0,1,0,0,1,0,0,0,0,0,0,0
2,87996131988367,650910917244495,0,1,0,0,0,0,0,0,0,0,0
3,87996127597456,650910884007885,1,1,1,0,0,0,0,0,0,0,0
4,87996140873613,650911013668532,1,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,87996140465712,0,1,0,0,1,0,0,0,0,0,0,0
96,87996131506300,0,1,0,0,1,0,0,0,0,0,0,0
97,87996134616994,650911151708071,1,0,1,0,1,0,0,0,0,0,0
98,87996129274824,650911219780570,1,1,1,0,0,0,0,0,0,0,0


In [19]:
order['driver_id'].replace('0', '-1', inplace=True)

In [20]:
order = order.astype('int64')

## buffer

In [22]:
states = feature.merge(mud,left_on=['f_real_order_id','waittime'],right_on=['order_id','waittime'],how='inner')

In [209]:
states.describe()

,f_real_order_id,f_order_id,nearby_total_driver_num,nearby_total_order_num,nearby_idle_driver_num,nearby_grid_order_num,in_total_driver_num,in_total_order_num,in_idle_driver_num,in_grid_order_num,...,decision_time,decision_dropoff2order_time,m_real_order_id,waittime,order_id,maxaction,max_qvalue,action,ymd,rn
count,4.034980e+05,4.034980e+05,403498.000000,403498.000000,403498.000000,403498.000000,403498.000000,403498.000000,403498.000000,403498.000000,...,403498.000000,403498.000000,4.034980e+05,403498.000000,4.034980e+05,403498.000000,403498.000000,403498.000000,4.034980e+05,403498.000000
mean,1.441721e+18,8.799613e+13,233.327966,13.887448,90.205282,3.879270,394.732892,32.214020,210.550462,32.214020,...,78.040860,11.545995,1.441721e+18,90.110598,1.441721e+18,1.182609,0.546980,1.167763,2.023110e+07,14.915402
std,7.787813e+11,1.154495e+07,112.303754,10.909285,50.055909,3.419011,123.395877,19.893353,78.921968,19.893353,...,147.552948,58.358929,7.787813e+11,100.527976,7.787813e+11,0.826488,0.167149,0.827749,4.988169e-01,22.355909
min,1.441719e+18,8.799611e+13,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,-1.000000,1.441719e+18,0.000000,1.441719e+18,0.003000,0.000000,0.003000,2.023110e+07,1.000000
25%,1.441719e+18,8.799612e+13,145.000000,6.000000,52.000000,1.000000,306.000000,17.000000,165.200000,17.000000,...,-1.000000,-1.000000,1.441719e+18,17.000000,1.441719e+18,0.412000,0.420683,0.412000,2.023110e+07,2.000000
50%,1.441721e+18,8.799613e+13,244.000000,11.000000,83.000000,3.000000,451.538462,29.000000,218.375000,29.000000,...,-1.000000,-1.000000,1.441721e+18,58.000000,1.441721e+18,0.992000,0.519009,0.992000,2.023110e+07,6.000000
75%,1.441721e+18,8.799614e+13,319.000000,19.000000,119.000000,5.000000,500.000000,43.000000,263.222222,43.000000,...,115.000000,-1.000000,1.441721e+18,126.000000,1.441721e+18,2.210000,0.661284,2.210000,2.023110e+07,17.000000
max,1.441721e+18,8.799615e+13,500.000000,75.000000,339.000000,26.000000,500.000000,141.000000,418.000000,141.000000,...,881.000000,596.000000,1.441721e+18,716.000000,1.441721e+18,2.210000,0.996324,2.210000,2.023110e+07,243.000000


In [23]:
states = states.assign(reward=0.0, done=0.0) # 增加了两列 states.shape (403498, 438)
states = states.rename(columns={'driver_id': 'states_driver_id'})

In [24]:
rewards = states[['b_real_order_id', 'f_order_id', 'states_driver_id', 'reward', 'done']].merge(order,left_on=['f_order_id', 'states_driver_id'],right_on=['order_id', 'driver_id'],how='left')
# 使用fillna函数将NaN值替换为-1  
rewards = rewards.fillna(-1)

In [25]:
rewards.shape

(498315, 18)

In [26]:
rewards.dtypes

b_real_order_id                                          int64
f_order_id                                               int64
states_driver_id                                         int64
reward                                                 float64
done                                                   float64
order_id                                               float64
driver_id                                              float64
calls                                                  float64
rides                                                  float64
acceptances                                            float64
calls_cancelled_before_accepted                        float64
calls_cancelled_by_passenger_after_accepted            float64
calls_cancelled_by_driver_after_accepted               float64
calls_cancelled_by_drivers_after_accepted_addresend    float64
serial_calls                                           float64
serial_rides                                           

In [27]:
sum([num for num in rewards.isna().sum()])

0

奖励替换

In [28]:
# 没有播单的，reward为0
rewards.loc[rewards['b_real_order_id'] == -1, 'reward'] = 0
rewards.loc[rewards['b_real_order_id'] == -1, 'done'] = 0

# 播单了但是没有司机接的，reward为-0.05
rewards.loc[(rewards['b_real_order_id'] != -1) & (rewards['driver_id'] != rewards['states_driver_id']), 'reward'] = -0.05
rewards.loc[(rewards['b_real_order_id'] != -1) & (rewards['driver_id'] != rewards['states_driver_id']), 'done'] = 0

# 乘客应答后取消，reward为0
rewards.loc[(rewards['calls_cancelled_by_passenger_after_accepted'] == 1) & (rewards['driver_id'] == rewards['states_driver_id']) & (rewards['driver_id'] != -1), 'reward'] = 0
rewards.loc[(rewards['calls_cancelled_by_passenger_after_accepted'] == 1) & (rewards['driver_id'] == rewards['states_driver_id']) & (rewards['driver_id'] != -1), 'done'] = 1

# 司机应答后取消，reward为0
rewards.loc[(rewards['calls_cancelled_by_driver_after_accepted'] == 1) & (rewards['driver_id'] == rewards['states_driver_id']) & (rewards['driver_id'] != -1), 'reward'] = 0
rewards.loc[(rewards['calls_cancelled_by_driver_after_accepted'] == 1) & (rewards['driver_id'] == rewards['states_driver_id']) & (rewards['driver_id'] != -1), 'done'] = 1

# 完单，reward为1
rewards.loc[(rewards['rides'] == 1) & (rewards['driver_id'] == rewards['states_driver_id']) & (rewards['driver_id'] != -1), 'reward'] = 1  
rewards.loc[(rewards['rides'] == 1) & (rewards['driver_id'] == rewards['states_driver_id']) & (rewards['driver_id'] != -1), 'done'] = 1

# 乘客应答前取消
rewards.loc[(rewards['calls_cancelled_before_accepted'] == 1) & (rewards['driver_id'] == rewards['states_driver_id']) & (rewards['driver_id'] != -1), 'reward'] = 0
rewards.loc[(rewards['calls_cancelled_before_accepted'] == 1) & (rewards['driver_id'] == rewards['states_driver_id']) & (rewards['driver_id'] != -1), 'done'] = 1

In [29]:
states['reward'] = rewards['reward']
states['done'] = rewards['done']

In [30]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [31]:
states['f_real_order_id'].value_counts()

f_real_order_id
1441719259658802220    200
1441719259643730991    172
1441721012001621482    160
1441719259663920680    158
1441721012023509385    153
                      ... 
1441721012027589404      1
1441719259661301289      1
1441721012028664113      1
1441721011999704242      1
1441721011991205985      1
Name: count, Length: 114634, dtype: int64

In [32]:
states[states['f_real_order_id'] == 1441721012008047462][['waittime', 'reward', 'done']].sort_values('waittime')

,waittime,reward,done
309519,1.0,1.0,0.0
23497,30.0,2.0,0.0
442209,44.0,1.0,0.0
34646,46.0,2.0,0.0
48585,73.0,1.0,0.0
24013,75.0,1.0,0.0
328056,77.0,1.0,0.0
99267,79.0,1.0,0.0
421336,81.0,1.0,0.0
479274,118.0,1.0,0.0


(s, a, r, s', done)

In [109]:
columns = [
    "f_real_order_id",
    "nearby_total_driver_num", 
    "nearby_total_order_num", 
    "nearby_idle_driver_num", 
    "nearby_grid_order_num", 
    "in_total_driver_num", 
    "in_total_order_num",
    "in_idle_driver_num",
    "in_grid_order_num",
    "pas_dynamic_times",
    "drv_dynamic_times",
    "waittime",
    "action",
    "reward",
    "done"
]


s = [
    "nearby_total_driver_num", 
    "nearby_total_order_num", 
    "nearby_idle_driver_num", 
    "nearby_grid_order_num", 
    "in_total_driver_num", 
    "in_total_order_num",
    "in_idle_driver_num",
    "in_grid_order_num",
    "pas_dynamic_times",
    "drv_dynamic_times",
    "waittime"
]

a = "action"
r = "reward"
d = "done"

In [34]:
states = states[columns]

In [102]:
sorted_states = states.groupby('f_real_order_id').apply(lambda x: x.sort_values('waittime')).reset_index(drop=True)

In [103]:
sorted_states.head(50)

,f_real_order_id,nearby_total_driver_num,nearby_total_order_num,nearby_idle_driver_num,nearby_grid_order_num,in_total_driver_num,in_total_order_num,in_idle_driver_num,in_grid_order_num,pas_dynamic_times,drv_dynamic_times,waittime,action,reward,done
0,1441719259641573292,208.0,10.0,54.0,4.0,202.0,17.0,90.0,17.0,0.0,0.0,233.0,0.003,1.0,0.0
1,1441719259641573292,136.0,14.0,30.0,5.0,207.0,15.0,91.0,15.0,0.0,0.0,248.0,0.161,1.0,0.0
2,1441719259641573292,149.0,13.0,41.0,6.0,230.0,22.0,92.0,22.0,0.0,0.0,262.0,2.210,1.0,0.0
3,1441719259641573292,149.0,13.0,41.0,6.0,206.0,21.0,96.0,21.0,0.0,0.0,264.0,0.161,1.0,0.0
4,1441719259641573292,257.0,12.0,61.0,5.0,217.0,20.0,102.0,20.0,0.0,0.0,279.0,0.992,1.0,0.0
5,1441719259641573292,126.0,13.0,35.0,4.0,206.0,20.0,91.0,20.0,0.0,0.0,293.0,2.210,1.0,0.0
6,1441719259641573292,126.0,13.0,35.0,4.0,210.0,23.0,94.0,23.0,0.0,0.0,295.0,0.992,1.0,0.0
7,1441719259641573292,131.0,15.0,37.0,4.0,211.0,21.0,97.0,21.0,0.0,0.0,297.0,0.992,1.0,0.0
8,1441719259641573292,131.0,15.0,37.0,4.0,208.0,21.0,99.0,21.0,0.0,0.0,299.0,0.992,1.0,0.0
9,1441719259641573292,131.0,15.0,37.0,4.0,236.0,22.0,98.0,22.0,0.0,0.0,301.0,0.003,1.0,0.0


In [104]:
states[s].max()

nearby_total_driver_num    500.0
nearby_total_order_num      75.0
nearby_idle_driver_num     339.0
nearby_grid_order_num       26.0
in_total_driver_num        500.0
in_total_order_num         141.0
in_idle_driver_num         424.0
in_grid_order_num          141.0
pas_dynamic_times            2.0
drv_dynamic_times            2.0
waittime                   716.0
dtype: float64

In [105]:
# 最大值归一化
sorted_states.loc[sorted_states[s].index, s] = sorted_states[s] / states[s].max()

In [106]:
sorted_states.head()

,f_real_order_id,nearby_total_driver_num,nearby_total_order_num,nearby_idle_driver_num,nearby_grid_order_num,in_total_driver_num,in_total_order_num,in_idle_driver_num,in_grid_order_num,pas_dynamic_times,drv_dynamic_times,waittime,action,reward,done
0,1441719259641573292,0.416,0.133333,0.159292,0.153846,0.404,0.120567,0.212264,0.120567,0.0,0.0,0.325419,0.003,1.0,0.0
1,1441719259641573292,0.272,0.186667,0.088496,0.192308,0.414,0.106383,0.214623,0.106383,0.0,0.0,0.346369,0.161,1.0,0.0
2,1441719259641573292,0.298,0.173333,0.120944,0.230769,0.460,0.156028,0.216981,0.156028,0.0,0.0,0.365922,2.210,1.0,0.0
3,1441719259641573292,0.298,0.173333,0.120944,0.230769,0.412,0.148936,0.226415,0.148936,0.0,0.0,0.368715,0.161,1.0,0.0
4,1441719259641573292,0.514,0.160000,0.179941,0.192308,0.434,0.141844,0.240566,0.141844,0.0,0.0,0.389665,0.992,1.0,0.0


In [107]:
states.shape

(498315, 15)

In [118]:
buffer = pd.DataFrame(columns=['id', 'wait', 's', 'a', 'r', 's_', 'd'], index=sorted_states.index)

buffer['id'] = sorted_states['f_real_order_id']
buffer['wait'] = sorted_states['waittime']
buffer['s'] = sorted_states[s].values.tolist()
buffer['a'] = sorted_states[a]
buffer['r'] = sorted_states[r]
buffer['s_'] = sorted_states[s].shift(-1).fillna(sorted_states[s].iloc[-1]).values.tolist()
buffer['d'] = sorted_states[d]

In [120]:
buffer['s_'].head()

0    [0.272, 0.18666666666666668, 0.088495575221238...
1    [0.298, 0.17333333333333334, 0.120943952802359...
2    [0.298, 0.17333333333333334, 0.120943952802359...
3    [0.514, 0.16, 0.17994100294985252, 0.192307692...
4    [0.252, 0.17333333333333334, 0.103244837758112...
Name: s_, dtype: object

In [169]:
buffer.drop(columns=['id', 'wait'], inplace=True)

In [155]:
# 使用条件索引找出当前行 id 不等于下一行 id 的行
mask = buffer['id'] != buffer['id'].shift(-1)
# 最后一行单独判断, 如果是done == 1保留，否则删除
if buffer.iloc[-1]['d'] == 1:
    mask[-1] = False
else:
    mask[-1] = True
buffer.drop(buffer[mask].index, inplace=True)

/tmp/ipykernel_1596370/153770438.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buffer.drop(buffer[mask].index, inplace=True)


In [162]:
buffer.loc[buffer['a'] == 0.003, 'a'] = 0
buffer.loc[buffer['a'] == 0.161, 'a'] = 1
buffer.loc[buffer['a'] == 0.412, 'a'] = 2
buffer.loc[buffer['a'] == 0.992, 'a'] = 3
buffer.loc[buffer['a'] == 1.583, 'a'] = 4
buffer.loc[buffer['a'] == 2.210, 'a'] = 5

In [164]:
buffer['a'].value_counts()

a
5.0    121116
4.0     90267
3.0     61363
2.0     50984
1.0     42787
0.0     42298
Name: count, dtype: int64

In [165]:
buffer.head()

,id,wait,s,a,r,s_,d
0,1441719259641573292,0.325419,"[0.416, 0.13333333333333333, 0.159292035398230...",0.0,0.0,"[0.272, 0.18666666666666668, 0.088495575221238...",0.0
1,1441719259641573292,0.346369,"[0.272, 0.18666666666666668, 0.088495575221238...",1.0,0.0,"[0.298, 0.17333333333333334, 0.120943952802359...",0.0
2,1441719259641573292,0.365922,"[0.298, 0.17333333333333334, 0.120943952802359...",5.0,0.0,"[0.298, 0.17333333333333334, 0.120943952802359...",0.0
3,1441719259641573292,0.368715,"[0.298, 0.17333333333333334, 0.120943952802359...",1.0,0.0,"[0.514, 0.16, 0.17994100294985252, 0.192307692...",0.0
4,1441719259641573292,0.389665,"[0.514, 0.16, 0.17994100294985252, 0.192307692...",3.0,0.0,"[0.252, 0.17333333333333334, 0.103244837758112...",0.0


In [170]:
buffer.to_csv('buffer.csv', sep='\t', index=False)

In [171]:
data_load = pd.read_csv('/home/zhx/word/DriverOrderOfflineRL/scripts/buffer.csv', sep='\t')

In [172]:
data_load.head()

,s,a,r,s_,d
0,"[0.416, 0.13333333333333333, 0.159292035398230...",0.0,0.0,"[0.272, 0.18666666666666668, 0.088495575221238...",0.0
1,"[0.272, 0.18666666666666668, 0.088495575221238...",1.0,0.0,"[0.298, 0.17333333333333334, 0.120943952802359...",0.0
2,"[0.298, 0.17333333333333334, 0.120943952802359...",5.0,0.0,"[0.298, 0.17333333333333334, 0.120943952802359...",0.0
3,"[0.298, 0.17333333333333334, 0.120943952802359...",1.0,0.0,"[0.514, 0.16, 0.17994100294985252, 0.192307692...",0.0
4,"[0.514, 0.16, 0.17994100294985252, 0.192307692...",3.0,0.0,"[0.252, 0.17333333333333334, 0.103244837758112...",0.0


In [174]:
buffer.describe()

,a,r,d
count,408815.000000,408815.000000,408815.000000
mean,3.168895,0.006958,0.018275
std,1.688747,0.131285,0.133943
min,0.000000,-0.050000,0.000000
25%,2.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000
75%,5.000000,0.000000,0.000000
max,5.000000,1.000000,1.000000
